In [1]:
%%time

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling
import seaborn as sns
import missingno as msno
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import json
import requests
from urllib.parse import urlencode

from surveymonkey.exceptions import UnknownError, BadRequestError, AuthorizationError, PermissionError, \
    ResourceNotFoundError, ResourceConflictError, RequestEntityTooLargeError, RateLimitReachedError, \
    InternalServerError, UserSoftDeletedError, UserDeletedError

'''
Token expiration and revocation
Our access tokens don’t currently expire but may in the future. We’ll warn all developers before making changes.
Access tokens can be revoked by the user. If this happens, you’ll get a JSON-encoded response body including a key 
statuswith a value of 1 and a key errmsg with the value of Client revoked access grant when making an API request. 
If you get this response, you’ll need to complete OAuth again.
'''

BASE_URL = "https://api.surveymonkey.com"
API_URL = "https://api.surveymonkey.com/v3"
AUTH_CODE = "/oauth/authorize"
ACCESS_TOKEN_URL = "/oauth/token"


class Client(object):

    def __init__(self, client_id=None, client_secret=None, redirect_uri=None, access_token=None):
        self.code = None
        self.client_id = client_id
        self.redirect_uri = redirect_uri
        self.client_secret = client_secret
        self._access_token = access_token

    def get_survey_lists(self):
        """
        List all created surveys.
        :return:
        """
        endpoint = "/surveys?per_page=100"
        url = API_URL + endpoint
        return self._get(url)

    def get_survey_details(self, survey_id):
        """
        Details of a specific survey
        :param survey_id: id of specific survey to get details from
        :return:
        """
        endpoint = f"/surveys/{survey_id}/details"
        url = API_URL + endpoint
        return self._get(url)

    def get_survey_response_bulk(self, survey_id, page):
        """
        Retrieves a list of full expanded responses, including answers to all questions.
        :param survey_id: id of survey
        :return:
        """
        endpoint = f"/surveys/{survey_id}/responses/bulk?page={page}&per_page=100"
        url = API_URL + endpoint
        return self._get(url)


    # Communications
    def _get(self, endpoint, **kwargs):
        return self._request('GET', endpoint, **kwargs)

    def _post(self, endpoint, **kwargs):
        return self._request('POST', endpoint, **kwargs)

    def _put(self, endpoint, **kwargs):
        return self._request('PUT', endpoint, **kwargs)

    def _patch(self, endpoint, **kwargs):
        return self._request('PATCH', endpoint, **kwargs)

    def _delete(self, endpoint, **kwargs):
        return self._request('DELETE', endpoint, **kwargs)

    def _request(self, method, url, **kwargs):
        _headers = {"Authorization": "Bearer %s" % self._access_token, "Content-Type": "application/json"}
        return self._parse(requests.request(method, url, headers=_headers, **kwargs))

    def _parse(self, response):
        status_code = response.status_code
        if 'application/json' in response.headers['Content-Type']:
            r = response.json()
        else:
            r = response.text
        if "error" in r:
            self.get_error(dict(r))
        if "error" not in r and status_code not in [200, 201, 204]:
            raise UnknownError()
        if status_code in (200, 201):
            return r
        if status_code == 204:
            return None

    def get_error(self, error):
        """
        :return:
        """
        error_code = error['error']
        error_message = error['message']
        if error_code == "1000":
            raise BadRequestError(error_message)
        elif error_code == "1001":
            raise BadRequestError(error_message)
        elif error_code == "1002":
            raise BadRequestError(error_message)
        elif error_code == "1003":
            raise BadRequestError(error_message)
        elif error_code == "1004":
            raise BadRequestError(error_message)
        elif error_code == "1010":
            raise AuthorizationError(error_message)
        elif error_code == "1011":
            raise AuthorizationError(error_message)
        elif error_code == "1012":
            raise AuthorizationError(error_message)
        elif error_code == "1013":
            raise AuthorizationError(error_message)
        elif error_code == "1014":
            raise PermissionError(error_message)
        elif error_code == "1015":
            raise PermissionError(error_message)
        elif error_code == "1016":
            raise PermissionError(error_message)
        elif error_code == "1017":
            raise PermissionError(error_message)
        elif error_code == "1018":
            raise PermissionError(error_message)
        elif error_code == "1020":
            raise ResourceNotFoundError(error_message)
        elif error_code == "1025":
            raise ResourceConflictError(error_message)
        elif error_code == "1026":
            raise ResourceConflictError(error_message)
        elif error_code == "1030":
            raise RequestEntityTooLargeError(error_message)
        elif error_code == "1040":
            raise RateLimitReachedError(error_message)
        elif error_code == "1050":
            raise InternalServerError(error_message)
        elif error_code == "1051":
            raise InternalServerError(error_message)
        elif error_code == "1052":
            raise UserSoftDeletedError(error_message)
        elif error_code == "1053":
            raise UserDeletedError(error_message)
        else:
            raise UnknownError("UNKNOWN ERROR: {}".format(error['message']))
            
ACCESS_TOKEN = 'your access token'
CLIENT_ID = 'your client id'
CLIENT_SECRET = 'your client secret token'
REDIRECT_URI = 'na'
client=Client(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, access_token=ACCESS_TOKEN)

def get_headings(json):
    
    '''
    This function takes a SurveyMonkey json object from an API call and creates a dataframe with the headers and the corresponding question_ids
    
    Arguments: a single json object created from a .get() call on the SurveyMonkey API
    '''
    
    #import necessary packages
    import pandas as pd
    import numpy as np
    
    ids_list = []
    for pages in json['pages']:
        for question in pages['questions']:
            for ids in question['id']:
                ids_list.append(ids)     
    def divide_chunks(l, n): 
      
    # looping till length l 
        for i in range(0, len(l), n):  
            yield l[i:i + n] 
  
    # How many elements each 
    # list should have 
    n = 9
  
    x = list(divide_chunks(ids_list, n)) 

    #create question id dataframe from list of ids the join is joining together each list into one number, taking out commas and quotation marks
    heading_ids = pd.DataFrame([''.join(i) for i in x])

    headings_list = []
    for pages in json['pages']:
        for question in pages['questions']:
            for headings in question['headings']:
                headings_list.append(headings)
                headings = pd.DataFrame(headings_list)

#concatenate the heading ids with the headings
    questions_and_ids = pd.concat([headings, heading_ids], axis = 1).rename(columns = {0:'question_id'})
    
    return pd.DataFrame(questions_and_ids)

def get_answers_and_ids(json):
    
    '''
    This function takes a json object from the SurveyMonkey API, flattens the json, subsets the items in the json, and extracts
    the answers and their corresponding ids and then places all of this into a new dataframe
    
    Arguments: json object from SurveyMonkey API call
    '''
    
    #flatten DEEPLY nested JSON, same source as above
    from itertools import chain, starmap

    def flatten_json_iterative_solution(dictionary):
        """Flatten a nested json file"""
        def unpack(parent_key, parent_value):
            """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
            if isinstance(parent_value, dict):
                for key, value in parent_value.items():
                    temp1 = parent_key + '_' + key
                    yield temp1, value
            elif isinstance(parent_value, list):
                i = 0 
                for value in parent_value:
                    temp2 = parent_key + '_'+str(i) 
                    i += 1
                    yield temp2, value
            else:
                yield parent_key, parent_value    

            
        # Keep iterating until the termination condition is satisfied
        while True:
            # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
            dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
            # Terminate condition: not any value in the json file is dictionary or list
            if not any(isinstance(value, dict) for value in dictionary.values()) and \
               not any(isinstance(value, list) for value in dictionary.values()):
                break

        return dictionary

    
    #use the function on the json
    flattened_details = pd.Series(flatten_json_iterative_solution(json)).to_frame().reset_index()
    
    flattened_details.rename(columns = {'index':'detail_buckets', 0:'details'}, inplace = True)
    
    
    #searching for all responses within the survey using regex
    choices = flattened_details[flattened_details['detail_buckets'].str.contains(r'questions_[0-9]{1,2}_answers_choices_[0-9]{1,2}_text|questions_[0-9]{1,2}_answers_other_text')].rename(columns = {'details':'possible_choices'})\
    .reset_index(drop = True)
    
    #searching for all response ids within the survey with regex
    choice_ids = flattened_details[flattened_details['detail_buckets'].str.\
    contains(r'pages_[0-9]{1,2}_questions_[0-9]{1,2}_answers_choices_[0-9]{1,2}_id|pages_[0-9]{1,2}_questions_[0-9]{1,2}_answers_other_id')].drop('detail_buckets', axis = 1).rename(columns = {'details':'response_id'}).reset_index(drop = True)

    final_df = pd.concat([choices, choice_ids], axis = 1)
    
    final_df.drop('detail_buckets', axis = 1, inplace = True)
    
    return final_df

#json = client.get_survey_response_bulk('186927358')
#json_2 = client.get_survey_details('186927358')

def get_personid_choiceid_rowid_surveyid(json, json_2):
    
    '''
    
    So, theres a lot going on here... basically this function takes a json object from the SurveyMonkey API call and produces all answers, 
    their corresponding row_ids (which are question ids in the surveys), the respondent ids, the question ids, choice ids, and text answers
    
    Arguments: you must create 2 json objects from the client call first, 
        json = bulk responses from API
        json_2 = survey details
        
    '''
    import pandas as pd
    import numpy as np
    
    #use function to flatten DEEPLY nested JSON, same source as above
    from itertools import chain, starmap

    def flatten_json_iterative_solution(dictionary):
        """Flatten a nested json file"""
        def unpack(parent_key, parent_value):
            """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
            if isinstance(parent_value, dict):
                for key, value in parent_value.items():
                    temp1 = parent_key + '_' + key
                    yield temp1, value
            elif isinstance(parent_value, list):
                i = 0 
                for value in parent_value:
                    temp2 = parent_key + '_'+str(i) 
                    i += 1
                    yield temp2, value
            else:
                yield parent_key, parent_value    

            
        # Keep iterating until the termination condition is satisfied
        while True:
            # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
            dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
            # Terminate condition: not any value in the json file is dictionary or list
            if not any(isinstance(value, dict) for value in dictionary.values()) and \
               not any(isinstance(value, list) for value in dictionary.values()):
                break

        return dictionary
    
    #getting the bulk responses and flattening the json file
    bulk_responses = pd.Series(flatten_json_iterative_solution(json)).to_frame().reset_index()
    
    #renaming the columns in the dataframe
    bulk_responses.rename(columns = {'index':'answer_type', 0:'answer'}, inplace = True)
    
    #searching for ids within the bulk responses, looks like I am only getting 50 unique back at a time...
    bulk_responses = bulk_responses[bulk_responses['answer_type'].str.contains(r'text|data_[0-9]{1,2}_id|data_[0-9]{1,2}_pages_[0-9]' \
    '{1,2}_questions_[0-9]{1,2}_id|data_[0-9]{1,2}_pages_[0-9]{1,2}_questions_[0-9]{1,2}_answers_[0-9]{1,2}_row_id|data_[0-9]{1,2}_pages_[0-9]' \
    '{1,2}_questions_[0-9]{1,2}_answers_[0-9]{1,2}_choice_id|other')].reset_index(drop = True)
    
    #grabbing the survey details from json_2
    survey_details = pd.Series(flatten_json_iterative_solution(json_2)).to_frame()
    
    #get survey id and create column denoting the survey id
    bulk_responses['survey_id'] = survey_details[survey_details.index.str.contains(r'^id') == True][0][0]

    #renameing the ids to normal names
    bulk_responses.replace(to_replace ='data_[0-9]{1,3}_id', value = 'respondent_id', regex = True, inplace = True)
    
    bulk_responses.replace(to_replace ='data_[0-9]{1,3}_pages_[0-9]{1,3}_questions_[0-9]{1,3}_answers_[0-9]{1,3}_text', 
                           value = 'text_answer', regex = True, inplace = True)
    
    bulk_responses.replace(to_replace ='data_[0-9]{1,3}_pages_[0-9]{1,3}_questions_[0-9]{1,3}_id', 
                           value = 'question_id', regex = True, inplace = True)
    
    bulk_responses.replace(to_replace ='data_[0-9]{1,3}_pages_[0-9]{1,3}_questions_[0-9]{1,3}_answers_[0-9]{1,3}_row_id', 
                           value = 'row_id', regex = True, inplace = True)
    
    bulk_responses.replace(to_replace ='data_[0-9]{1,3}_pages_[0-9]{1,3}_questions_[0-9]{1,3}_answers_[0-9]{1,3}_choice_id', 
                           value = 'choice_id', regex = True, inplace = True)
    
    #some surveys have "other" as an option, this covers those
    bulk_responses.replace(to_replace ='data_[0-9]{1,4}_pages_[0-9]{1,3}_questions_[0-9]{1,3}_answers_[0-9]{1,3}_other_id', 
                           value = 'other_id', regex = True, inplace = True)

    #fill in survey_id column completely
    bulk_responses.ffill(inplace = True)
    
    #create mask to use as a transfer from row ids to another column
    mask = (bulk_responses['answer_type'] == 'row_id')

    #use the mask
    bulk_responses['row_id'] = bulk_responses['answer_type'][mask]

    #set row id equal to the actual row id
    bulk_responses.row_id[bulk_responses.row_id == 'row_id'] = bulk_responses.answer

    #shift all row ids up one in order to drop the row ids from the details column
    bulk_responses['row_id'] = bulk_responses['row_id'].shift(-1)

    #drop all row ids rows from df so that row id is listed beside choice id
    bulk_responses = bulk_responses[~bulk_responses.answer_type.str.contains('row_id')]

    #rearrange columns
    bulk_responses = bulk_responses[['answer_type', 'answer', 'row_id', 'survey_id']]
    
    #create mask to use as a transfer to question ids column
    mask_2 = (bulk_responses['answer_type'] == 'question_id')

    #use the mask
    bulk_responses['question_id'] = bulk_responses['answer_type'][mask_2]

    #put actual question ids into the question ids column
    bulk_responses.question_id[bulk_responses.question_id == 'question_id'] = bulk_responses.answer

    #shift all of them down 1 to make sure answers line up next to actual questions
    bulk_responses['question_id'] = bulk_responses['question_id'].shift(1)

    #drop question id from answer type
    bulk_responses = bulk_responses[~bulk_responses.answer_type.str.contains('question_id')]

    #forward fill the question ids so that each row id has a corresponding question id
    bulk_responses['question_id'] = bulk_responses['question_id'].ffill()

    mask_3 = (bulk_responses['answer_type'] == 'respondent_id')

    bulk_responses['respondent_id'] = bulk_responses['answer_type'][mask_3]

    bulk_responses.respondent_id[bulk_responses.respondent_id == 'respondent_id'] = bulk_responses.answer

    bulk_responses['respondent_id'] = bulk_responses['respondent_id'].ffill()

    bulk_responses = bulk_responses[~bulk_responses['answer_type'].str.contains('respondent_id')]
    
    #change column order
    bulk_responses = bulk_responses[['respondent_id', 'survey_id', 'answer_type', 'answer', 'row_id', 'question_id']].reset_index(drop = True)
    
    return bulk_responses

# = client.get_survey_details('186925932')

def get_row_text_and_row_ids(json):
    
    '''
    This function takes a json object returned from a call to the SurveyMonkey API and returns sub-questions and their ids
    
    Argument: a json object called from the SurveyMonkey API
    
    Requirements: the SurveyMonkey client - https://github.com/GearPlug/surveymonkey-python
    '''
    
    from itertools import chain, starmap

    def flatten_json_iterative_solution(dictionary):
        """Flatten a nested json file"""
        def unpack(parent_key, parent_value):
            """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
            if isinstance(parent_value, dict):
                for key, value in parent_value.items():
                    temp1 = parent_key + '_' + key
                    yield temp1, value
            elif isinstance(parent_value, list):
                i = 0 
                for value in parent_value:
                    temp2 = parent_key + '_'+str(i) 
                    i += 1
                    yield temp2, value
            else:
                yield parent_key, parent_value    

            
        # Keep iterating until the termination condition is satisfied
        while True:
            # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
            dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
            # Terminate condition: not any value in the json file is dictionary or list
            if not any(isinstance(value, dict) for value in dictionary.values()) and \
               not any(isinstance(value, list) for value in dictionary.values()):
                break

        return dictionary
    
    
    flattened_details = pd.Series(flatten_json_iterative_solution(json)).to_frame().reset_index()
    
    flattened_details.rename(columns = {0:'row_id'}, inplace = True)
    
    row_text = flattened_details[flattened_details['index'].str.contains(r'pages_[0-9]{1,4}_questions_[0-9]{1,4}_answers_rows_[0-9]{1,4}_text|rows_[0-9]{1,4}_id')] 
    
    row_text_ids = row_text[row_text['row_id'].str.contains(r'^[0-9]{1,9}') == True]

    row_id_df = pd.DataFrame(pd.concat([row_text, row_text_ids], axis = 1)['row_id'].iloc[:,1].dropna())
    
    row_text_df =  pd.DataFrame(pd.concat([row_text, row_text_ids], axis = 1)['row_id'].iloc[:,0].dropna())

    rows_df = pd.concat([row_text_df, row_id_df], axis = 1)

    rows_df.iloc[:,1] = rows_df.iloc[:,1].shift(-1)

    rows_df.dropna(inplace = True)

    rows_df.columns = ['row_text', 'row_id']
    
    return rows_df.reset_index(drop = True)


# -----------------------------------------------------------------------get all surveys


surveys = pd.Series(flatten_json_iterative_solution(client.get_survey_lists())).to_frame()
surveys.reset_index(inplace = True)
surveys.rename(columns = {'index':'detail_buckets', 0:'details'}, inplace = True)
surveys = surveys[surveys['detail_buckets'].str.contains('data_[0-9]{1,20}_id|data_[0-9]{1,20}_title')]
mask = surveys.details.str.contains(r'[A-Za-z]')
surveys['survey_name'] = surveys['details'][mask]
surveys['survey_name'] = surveys['survey_name'].shift(-1)
surveys = surveys[~surveys['detail_buckets'].str.contains(r'data_[0-9]{1,20}_title')]
surveys.rename(columns = {'details':'survey_id'}, inplace = True)
surveys.drop('detail_buckets', axis = 1, inplace = True)
surveys.reset_index(drop = True, inplace = True)

#--------------------------------------------------------------------------------get all headings


all_headings = pd.concat([get_headings(client.get_survey_details(survey_id)) for survey_id in surveys['survey_id']])

#-------------------------------------------------------------------------------get all possible responses


every_single_possible_response = pd.concat([get_answers_and_ids(client.get_survey_details(survey_id)) for survey_id in surveys['survey_id']])

#----------------------------------------------------------------------------------get all subquestions


every_single_subquestion_info = pd.concat([get_row_text_and_row_ids(client.get_survey_details(survey_id)) for survey_id in surveys['survey_id']])

#-------------------------------------------------------------------------------------getting all actual responses

#looping through every single survey and, depending on the number of responses to that survey, looping through each subsequent page of responses
# and grabbing every single answer from that survey


all_responses = []
for survey_id in surveys['survey_id']:
    if client.get_survey_details(survey_id)['response_count'] <= 100:
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 1), client.get_survey_details(survey_id)))
    elif client.get_survey_details(survey_id)['response_count'] in range(101, 200):
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 1), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 2), client.get_survey_details(survey_id)))
    elif client.get_survey_details(survey_id)['response_count'] in range(201, 300):
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 1), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 2), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 3), client.get_survey_details(survey_id)))
    elif client.get_survey_details(survey_id)['response_count'] in range(301, 400):
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 1), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 2), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 3), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 4), client.get_survey_details(survey_id)))
    elif client.get_survey_details(survey_id)['response_count'] in range(401, 500):
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 1), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 2), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 3), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 4), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 5), client.get_survey_details(survey_id)))
    elif client.get_survey_details(survey_id)['response_count'] in range(501, 600):
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 1), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 2), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 3), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 4), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 5), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 6), client.get_survey_details(survey_id)))
    elif client.get_survey_details(survey_id)['response_count'] in range(601, 700):
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 1), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 2), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 3), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 4), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 5), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 6), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 7), client.get_survey_details(survey_id)))
    elif client.get_survey_details(survey_id)['response_count'] in range(701, 800):
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 1), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 2), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 3), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 4), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 5), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 6), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 7), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 8), client.get_survey_details(survey_id)))
    elif client.get_survey_details(survey_id)['response_count'] in range(801, 900):
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 1), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 2), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 3), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 4), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 5), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 6), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 7), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 8), client.get_survey_details(survey_id)))
        all_responses.append(get_personid_choiceid_rowid_surveyid(client.get_survey_response_bulk(survey_id, 9), client.get_survey_details(survey_id)))

#concatenate the list into one dataframe
all_responses = pd.concat(all_responses)

#--------------------------------------------------------------------------------------------connect to SQL


from sqlalchemy import create_engine
import pymysql

#create the engine
db_connection_str = 'mysql+pymysql://[your username (usually root)]:[your password]@[your hostname (usually 127.0.0.1)]/[schema that you want to connect to/send df into]'
db_connection = create_engine(db_connection_str)

#---------------------------------------------------------------------------------------------clean dfs for SQL export

all_headings = all_headings[['question_id', 'heading']]
#clear out anything but numbers and letters
all_headings['heading'] = [re.sub(r'\W+', ' ', str(question)) for question in every_single_heading['heading']]

all_responses.rename(columns = {'index':'rID'}, inplace = True)
#clear out anything but numbers and letters
all_responses['answer'] = [re.sub(r'\W+', ' ', str(answer)) for answer in all_responses['answer']]

#-----------------------------------------------------------------------------------send it all to SQL (you must set up your schema in workbench first)
# all_responses.to_sql('responses', con = db_connection, if_exists = 'replace', index = False)
# surveys.to_sql('surveys', con = db_connection, if_exists = 'replace', index = False)
# every_single_heading.to_sql('questions', con = db_connection, if_exists = 'replace', index = False)
# every_single_possible_response.to_sql('possible_responses', con = db_connection, if_exists = 'replace', index = False)
# every_single_subquestion_info.to_sql('subquestions', con = db_connection, if_exists = 'replace', index = False)